### Installation:

pip install -U -q earthenginge-api

pip install -U -q folium

pip install matplotlib

### Initialization

In [59]:
import ee
from IPython.display import Image
import folium
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'matplotlib'

### Authentication:

In [ ]:
ee.Authenticate()
ee.Initialize()

Will generate a token to kennect with gcloud on GEE:

Implementing this will require a GEE account and repo setup to use

##### Test Imports and Auth wokred

In [56]:
#Test
# Print the elevation of Mount Everest.
dem = ee.Image('USGS/SRTMGL1_003')
xy = ee.Geometry.Point([86.9250, 27.9881])
elev = dem.sample(xy, 30).first().get('elevation').getInfo()
print('Mount Everest elevation (m):', elev)

Mount Everest elevation (m): 8729


In [ ]:
# Import a DEM and display a thumbnail of it.
dem = ee.Image('USGS/SRTMGL1_003')
Image(url=dem.updateMask(dem.gt(0))
  .getThumbUrl({'min': 0, 'max': 4000, 'dimensions': 512,
                'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']}))

In [ ]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)


Siboniso: surface albedo

In [ ]:
#/ Define the region of interest (South Africa) 
roi = ee.Geometry.Rectangle([16.3, -34.8, 32.9, -22.1]); 
 
#// Define the date range 
startDate = '2023-01-01'; 
endDate = '2023-01-31'; 

#// Load MODIS albedo data 
modisAlbedo = ee.ImageCollection('MODIS/006/MCD43A3').filterDate(startDate, endDate).select('Albedo_BSA_shortwave'); 

#// Calculate the median albedo for the date range 
medianAlbedo = modisAlbedo.median(); 

#// Clip the image to the region of interest 
clippedAlbedo = medianAlbedo.clip(roi); 


#Add the clipped albedo image to the map 

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer
# Set visualization parameters.
vis_params = {
  'min': 0,
  'max': 5000,
  'palette': ['006633', 'E5FFCC', '662A00', 'D8D8D8', 'F5F5F5']
  #'palette': ['FFFFFF, CE7E45, DF923D, F1B555, FCD163, 99B718,'+
  #            '74A901, 66A000, 529400, 3E8601, 207401, 056201,'+
  #            '004C00, 023B01, 012E01, 011D01, 011301']
  }


# Create a folium map object.
my_map = folium.Map(location=[-30, 21], zoom_start=6)

# Add the elevation model to the map object.
my_map.add_ee_layer(clippedAlbedo, vis_params, 'Surface Albedo') 

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)
